## Introduction
In this demo, I'll be showing you how to write some basic proofs using Pylogic. First, a brief description of Pylogic.

Pylogic is a proof assistant written entirely in Python. It uses structural pattern matching to check whether inference rules
are correctly applied, in which case an inference can be derived.

This is not an automated theorem prover, and cannot prove most complex statements by itself. How to use it is to write out the proof
steps as you would by hand, but using Pylogic methods instead.

Pylogic is still in early development, and there are still many bugs to be fixed. However, the inference rules themselves are
logically valid.

In this demo, if you encounter any errors or seemingly wrong outputs, try running all cells from the top again to ensure everything is initialized correctly. 

This demo might seem long, but it's not meant to be. There's a lot of explanatory text and code. Feel free to skip some parts if you are familiar with the concepts.

Feel free to play around with the code, such as checking the `.inference_rules` attribute to see the list of inference rule methods a proposition or class supports. For each inference rule, if you call the `help` function on it, you should get a little (hopefully helpful but may not currently be)
docstring explaining it.

An example is shown in at the end of the demo.

## Pylogic
There are two main types of objects in Pylogic: `Proposition`s and `Term`s. A `Proposition` represents a mathematical sentence that
can be proven, or its negation can be proven. This is similar to saying the `Proposition` is true or false, but is slightly different.

A `Proposition` that is not proven is not necessarily False, we just have not shown if it (or its negation) is True.

`Terms` are objects that can serve as arguments to `Proposition`s. A `Term` is something like a set, a number, a sequence, etc. For example,
in the code

```python
x.is_in(S)
```
which represents a sentence like $x \in S$ where $x$ is a number and $S$ is a set, $x$ and $S$ are `Term`s, and $x \in S$ is a `Proposition`.

Typically, all the terms in a `Proposition` are visible as its arguments under the `.args` attribute.

In Pylogic, there are `Variable`, `Constant`, `Set`, `Sequence`, and expressions like `Add`, `Mul`, etc representing different types of `Term`s.

## Proofs
In a proof, you typically start with some assumptions and derive some conclusion. Let's first create a proposition. We can create an atomic
proposition named P as follows

In [1]:
from pylogic.proposition.proposition import Proposition

P = Proposition('P')

`P` has no arguments, which we see when we access `P.args`

In [2]:
P.args

[]

This could represent something like "It rains today", if we assume that has no arguments.

Let's create another proposition with an argument,
called `Qx`.

In [3]:
from pylogic.proposition.proposition import Proposition
from pylogic.constant import Constant
x = Constant('x')
Qx = Proposition('Q', args=[x])
Qx

Proposition(Q, x)

Here, `x` is the argument. We had to create a `Constant` term as the argument. In future iterations, this will be simplified.

`x` represents some term, such as "the ground" in the sentence "the ground is wet". We can imagine the `Q` to be a 'predicate', something
that needs an argument `x` to become a Proposition. Just like "__ is wet" needs an argument to make sense.

Of course, we could have named the propositions anything, and this will show up when we view it in the notebook. Note: if you type a value
(without printing) or use the `display` function, the Ipython (Jupyter) notebook shows a latex representation of the object if available.

Now what if we want the statement "If it rains today, the ground is wet"? We can create such as statement using `.implies`:

In [4]:
P.implies(Qx)

Implies(Proposition(P), Proposition(Q, x))

The implication is displayed as antecedent $\rightarrow$ consequent. We just created these 3 propositions `P`, `Qx` and `P -> Qx`, which we did not yet save to a variable. Let's do that, so we can refer to it.

In [5]:
P_implies_Qx = P.implies(Qx)
P_implies_Qx

Implies(Proposition(P), Proposition(Q, x))

Now, each of these are just statements, but we're not doing anything with them. We haven't assumed anything, and we can't prove anything.

Let's make some assumptions. Suppose we know that "If it rains today, the ground is wet" to be true, and we also know "It rains today" to be true.
Since we already created these propositions, we can simply assume them, and they are marked as assumptions.

Before we assume them, let's take a look at the `.is_assumption` and `.is_proven` attributes

In [6]:
print(P.is_assumption)
print(P_implies_Qx.is_assumption)

print(P.is_proven)
print(P_implies_Qx.is_proven)

False
False
False
False


In [7]:
P.assume()
P_implies_Qx.assume()

print(P.is_assumption)
print(P_implies_Qx.is_assumption)
print(P.is_proven)
print(P_implies_Qx.is_proven)

True
True
True
True


Now these are assumed to be true. Because they are assumed, they have automatically been proven.

The goal of a proof is to get the precise sentence you want to prove, with the `.is_proven` attribute as `True`.

Of course, just assuming what you want to prove is trivial and not helpful. Here, we can try to prove something new.
Let's prove that the ground is wet, using our assumptions.

We can use the _Modus ponens_ inference rule to do this. _Modus ponens_ says that given "A" and "A implies B" as premises, we can conclude "B".

There are a couple of ways to use _modus ponens_ here, but the straight-forward way is to call it on the first premise, the "A" in ("A" and "A implies B"), passing the second premise ("A implies B") which is an implication, as an argument.

In this case, we do the following:

In [8]:
P.modus_ponens(P_implies_Qx)

Proposition(Q, x)

We see that we get Qx. Whenever we use an inference rule method, it typically produces a proven proposition if all the premises and argument
are logically valid, and the structures match appropriately.

Let's save the result of modus_ponens in a variable and inspect it.

In [9]:
conclusion = P.modus_ponens(P_implies_Qx)
display(conclusion)
print(conclusion.is_proven)

Proposition(Q, x)

True


We see that the conclusion is indeed `Qx` and it is proven. Here is the same proof but in more natural language:

In [10]:
It_is_raining = Proposition('It is raining')
the_ground = Constant('the ground')
the_ground_is_wet = Proposition('is wet', args=[the_ground])

It_is_raining.assume()
If_it_is_raining_then_the_ground_is_wet = It_is_raining.implies(the_ground_is_wet).assume()
conclusion = It_is_raining.modus_ponens(If_it_is_raining_then_the_ground_is_wet)
display(conclusion)
print(conclusion.is_proven)

Proposition(is wet, the ground)

True


Let's do another proof at at faster pace. We will prove a conjunction by proving each of its conjuncts, the sentences that make it up.

First, we will assume a universally quantified sentence "for all x, P(x)" is true. Then we assume "Q(2)", and "Q(2) implies R(2)" are true.

We want to prove that "P(2) and R(2)" is true.

For quantification, we need to use variables.

In [11]:
from pylogic.variable import Variable
from pylogic.proposition.quantified.forall import Forall

x = Variable('x')
forall_x_px = Forall(x, Proposition('P', args=[x]), is_assumption=True)
forall_x_px

Forall(Variable(x, deps=()), Proposition(P, x))

`Forall` is a subclass of `Proposition`, used to represent a universally quantified sentence such as the one above. It means,
"P(x) is true for every x".

Above, I have made `forall_x_px` an assumption directly when I created it by setting `is_assumption=True`. Note that `is_assumption=True` is set on the outermost proposition, the `Forall`, not the inner proposition, `P(x)`.

We can do similarly for our other assumptions.

In [12]:
from pylogic.constant import Constant
two = Constant(2)
Q2 = Proposition('Q', args=[two], is_assumption=True)
R2 = Proposition('R', args=[two])

Q2_implies_R2 = Q2.implies(R2, is_assumption=True)

Now we have three assumptions. Observe how they are made assumptions by using the `is_assumption` keyword argument directly on the outermost proposition.
In the case of `Q2_implies_R2`, the `.implies` method creates an implication, and we can pass other keyword arguments such as `is_assumption`
to build the resulting proposition.

Another way to create the implication is below:

In [13]:
from pylogic.proposition.implies import Implies
Q2_implies_R2 = Implies(Q2, R2, is_assumption=True)
Q2_implies_R2

Implies(Proposition(Q, 2), Proposition(R, 2))

Also note that we created proposition `R2` above, but we did not assume it. We also haven't proven it yet. It was simply created
to help us subsequently create the proposition `Q2_implies_R2`, which we did assume. If you were to inspect the `.is_proven` method
on `R2` above, you would get False:

In [14]:
R2.is_proven

False

Now we can use _modus ponens_ and our assumption `Q2` to prove `R2`:

In [15]:
R2 = Q2.modus_ponens(Q2_implies_R2)
display(R2)
print(R2.is_proven)

Proposition(R, 2)

True


Remember that we wanted to prove that "P(2) and R(2)" is true. We're halfway; let's prove "P(2)" using the universally quantified sentence.

Since "for all x, P(x)" is true, P should be true in particular for 2, since 2 is an object in our universe.
We use an `.in_particular` method to achieve this:

In [16]:
P2 = forall_x_px.in_particular(two)
display(P2)
print(P2.is_proven)

Proposition(P, 2)

True


Compare the universally quantified sentence `forall_x_px` above to `P2`. The term `2` was substituted for every appearance of variable `x` in the `Forall` sentence.

Now that we have `P2` and `R2` individually, we can prove the conjunction `P2 and R2`:

In [17]:
P2_and_R2 = P2.and_(R2)
display(P2_and_R2)
print(P2_and_R2.is_proven)

And(Proposition(P, 2), Proposition(R, 2))

True


`.and_` is also an inference rule: if we have proven each individual part, we can prove a conjunction made of these parts.

We reached our goal, and we're happy 🎉

## Outro

A few things to note. There is an underscore at the end of `.and_`. This is because Python already uses `and` as a keyword. If you have suggestions
for better names for some of these inference rules, I'd be happy to hear them.

Also, you don't have to inspect the `.is_proven` attribute every time; it was done here to show how the propositions behave
with this attribute when assumed, proven or neither. If you are sure the inference worked, which is when there are no errors,
you can move on to your next goal.

You can take a look at all the inference rules available on a proposition by looking at the `.inference_rules` attribute. For each inference rule, call the `help` function to get its docstring.

In [18]:
Q2_implies_R2.inference_rules

['hypothetical_syllogism',
 'impl_elim',
 'definite_clause_resolve',
 'unit_definite_clause_resolve',
 'first_unit_definite_clause_resolve',
 'contrapositive']

In [19]:
help(Q2_implies_R2.unit_definite_clause_resolve)

Help on method unit_definite_clause_resolve in module pylogic.proposition.implies:

unit_definite_clause_resolve(in_body: 'Proposition') -> 'Self | Implies[Proposition, UProposition] | UProposition' method of pylogic.proposition.implies.Implies instance
    Logical inference rule. Given self `(A /\ B /\ C...) -> D` is proven, and
    given one of the props (say B) in the antecedent is proven,
    return a proof of the new definite clause `(A /\ C /\ ...) -> D`
    or `A -> D` if only A is left in the body, or D if the antecedent is
    left empty.



For most inference rules, you will need **proven** propositions (either by assuming them or proving them from other assumptions) to work.
The structures of the inference rules must match as specified.

If the docstrings are not helpful, as they are still a work in progress, you can look-up some of the inference rule names online.

## Conclusion

This was a brief introduction to Pylogic, a mathematical proof assistant in Python.
Much work remains to be done, such as simplifying import statements and adding convenience functions.

For now, the following imports are needed if you want to work with the corresponding classes and terms below.

In [20]:
from pylogic.proposition.proposition import Proposition
from pylogic.proposition.and_ import And
from pylogic.proposition.or_ import Or
from pylogic.proposition.not_ import Not
from pylogic.proposition.implies import Implies
from pylogic.constant import Constant
from pylogic.variable import Variable
from pylogic.structures.set_ import Set
from pylogic.proposition.quantified.forall import Forall, ForallInSet, ForallSubsets
from pylogic.proposition.quantified.exists import Exists, ExistsInSet, ExistsSubset, ExistsUnique, ExistsUniqueInSet, ExistsUniqueSubset
from pylogic.structures.grouplike.group import Group, AbelianGroup
from pylogic.structures.ringlike.ring import RIng
from pylogic.structures.ringlike.rng import Rng
from pylogic.structures.ringlike.field import Field
from pylogic.theories.real_analysis import Reals
from pylogic.theories.natural_numbers import Naturals
from pylogic.abc import x, y, z # variables
from pylogic.abc import cx, cy, cz # constants
from pylogic.assumptions_context import AssumptionsContext
from pylogic.expressions.abs import Abs
from pylogic.proposition.ordering.lessthan import LessThan
from pylogic.proposition.ordering.greaterorequal import GreaterOrEqual
from pylogic.proposition.relation.equals import Equals
from pylogic.proposition.relation.contains import IsContainedIn

In [21]:
Reals

Set_Reals

In [22]:
x.is_in(Naturals)

IsContainedIn(x, Naturals)

Thanks for your attention!